In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/MyDrive/Phase_Retrieval/classification_over_measurements/utils")

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [ ]:
#@title Parámetros
SHAPE = (128,128)
BATCH_SIZE = 512
NUM_MEASUREMENTS = 1
NUM_CLASSES = 10

dataset = "cifar10" # mnist - fashion_mnist - cifar10
initialization = "None" # None - backpropagation - FSI

In [ ]:
results_folder = os.path.join("..", "results", dataset)
if not os.path.exists(results_folder):
  os.mkdir(results_folder)
results_folder = os.path.join(results_folder, initialization)
if not os.path.exists(results_folder):
  print("Creado")
  os.mkdir(results_folder)

In [ ]:
def preprocess_cifar(image,label, shape, num_clases):
  x = tf.math.reduce_mean(image, axis=-1, keepdims=True)
  x = tf.image.resize(x, shape)
  

  mins = tf.math.reduce_min(x,axis=[1,2], keepdims=True)
  maxs = tf.math.reduce_max(x,axis=[1,2], keepdims=True)
  x = tf.math.divide(tf.math.subtract(x,mins),tf.math.subtract(maxs,mins))

  label = tf.one_hot(tf.cast(label, tf.int32), num_clases)
  return x,label

In [ ]:
train_images = tfds.load(dataset, split='train', as_supervised=True, batch_size=BATCH_SIZE)
test_images = tfds.load(dataset, split='test', as_supervised=True, batch_size=BATCH_SIZE)

In [ ]:
train_images = train_images.map(lambda x,y: preprocess_cifar(x,y, shape= SHAPE, num_clases=10))
test_images = test_images.map(lambda x,y: preprocess_cifar(x,y, shape= SHAPE, num_clases=10))

In [ ]:
modelo = tf.keras.applications.xception.Xception(include_top=True, input_shape=(*SHAPE, 1), pooling="avg", weights=None, classes=10,classifier_activation='softmax')

In [ ]:
#modelo.summary()

In [ ]:
num_epochs = 50
chekpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join(results_folder, "pesos_inception"+"_"+dataset+".h5"), save_weights_only=True)
opti = tf.keras.optimizers.Adam(amsgrad=True, learning_rate=1e-4)
modelo.compile(optimizer=opti, loss = "binary_crossentropy", metrics=[tf.keras.metrics.BinaryAccuracy(name="binary_accuracy"), tf.keras.metrics.Recall(name = "recall_1"), tf.keras.metrics.Precision(name = "precision_1")])
history = modelo.fit(train_images, epochs=num_epochs,validation_data=test_images, callbacks=[chekpoint])

Epoch 1/50
  1/391 [..............................] - ETA: 4:45:30 - loss: 0.6388 - binary_accuracy: 0.9000 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00

KeyboardInterrupt: ignored